In [ ]:
%pylab inline
from textgen import *

m = TextGen(
    img_size=299, 
    code_dim=512, 
    rnn_type='LSTM', 
    cnn_type='InceptionV3',
    pooling='avg',
)

In [ ]:
def validate(model, val_record, ckpt, batch_size=32, capacity=8, n_threads=6, epoch_size=10000, 
             captions_json='/home/paperspace/data/ms_coco/captions_val2014.json',
             inds_to_words_json='/home/paperspace/data/ms_coco/SOS_preproc_vocab-9413_threshold-5_length-20/inds_to_words.json'):
    ''''''
    with tf.variable_scope('BatchReader'):
        print('creating batch reader')
        coord =  tf.train.Coordinator()

        # read and preprocess training records
        x, y, _ = model.read_tfrecord(
            val_record, 
            batch_size=batch_size, 
            capacity=capacity, 
            n_threads=n_threads,
            n_epochs=None)
        x = model.preproc_img(x)
        y, _ = model.preproc_caption(y, random=False)

    with tf.variable_scope('Strings'):
        print('getting strings')

        y_hat = model.sample_recursively(x, continuous=False)
        table = model.create_table(inds_to_words_json)
        y_hat_strings = model.postproc_caption(y_hat, table)[0]

    with tf.Session(graph=model.graph) as sess:
        print('starting threads')
        tf.train.start_queue_runners(sess=sess, coord=coord)

        print('initializing variables')
        sess.run(tf.global_variables_initializer())

        print('loading weights from '+ckpt)
        model.load_weights(ckpt)
        
        print('loading IDs from '+captions_json)
        ids = sorted([e['id'] for e in \
                json.loads(open(captions_json)\
                   .read())['images']])

        n = 0
        results = []
        try:
            for _ in tqdm.trange(epoch_size):
                if not coord.should_stop():
                    s = sess.run(y_hat_strings, {backend.learning_phase(): False})
                    id_ = ids[n]
                    for i, c in enumerate(s):
                        d = {"image_id": id_[i], "caption": c}
                    results.append(d)
                    n += 1
                else:
                    raise IOError, 'queues closed'
            return results
        # exit behaviour: request thread stop, then wait for 
        # them to recieve message before exiting session context
        except:
            coord.request_stop()
            time.sleep(0.2)
            raise

In [ ]:
results = validate(m, 
 '/home/paperspace/data/ms_coco/SOS_preproc_vocab-9413_threshold-5_length-20/val_1shot.tfrecord',
 '/home/paperspace/training/1shot/lr4e-4_decay100k_bs16_reg1e-4_clip50_baseclasses/ckpt_update-0000890000.h5', 
batch_size=32)

In [ ]:
sorted([e['id'] for e in \
        json.loads(open('/home/paperspace/data/ms_coco/captions_val2014.json')\
                   .read())['images']])